In [7]:
import pandas as pd
import json
import os
from shapely.geometry import Polygon
from pyproj import Proj, transform, CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import geopandas as gpd
from pathlib import Path

In [8]:
DATA_DIR = Path('data')
MERGED_DIR = DATA_DIR / 'merged'
CONFIG_FILE = Path('config.json')

with open(CONFIG_FILE, 'r') as f:
    config = json.load(f)

IMG_SIZE = config["MODIS"]["image_size_km"]
SITES = os.listdir(MERGED_DIR)

In [9]:
def load_meta(site):
    with open(MERGED_DIR/site/'meta.json', 'r') as f:
        return json.load(f)

data = [load_meta(s) for s in SITES]
meta_df = pd.DataFrame(data)

In [11]:
def get_polygon_wkt(lat, lon):
    utm_crs_info = query_utm_crs_info(
        area_of_interest=AreaOfInterest(west_lon_degree=lon, south_lat_degree=lat, east_lon_degree=lon, north_lat_degree=lat),
        datum_name="WGS 84"
    )[0]
    utm_crs = CRS.from_epsg(utm_crs_info.code)

    # Projected coordinate system for accurate distance measurement
    proj_utm = Proj(utm_crs)

    # Convert the given lat/lon in WGS84 to UTM coordinates
    x, y = proj_utm(lon, lat)
    offset = IMG_SIZE * 1000 / 2  # number of meters in each direction

    # Calculate the corners of the box in UTM coordinates
    bottom_left = (x - offset, y - offset)
    bottom_right = (x + offset, y - offset)
    top_right = (x + offset, y + offset)
    top_left = (x - offset, y + offset)

    # Create a polygon from these corners
    box = Polygon([bottom_left, bottom_right, top_right, top_left, bottom_left])

    # Optionally, convert the polygon back to geographic coordinates (WGS84)
    proj_wgs84 = Proj(proj='latlong', datum='WGS84')
    lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)

    # Create a new polygon in geographic coordinates
    geo_polygon = Polygon(zip(lon_lat_polygon[0], lon_lat_polygon[1]))

    return geo_polygon

meta_df['geometry'] = meta_df.apply(lambda row: get_polygon_wkt(row['LOCATION_LAT'], row['LOCATION_LON']), axis=1)


/tmp/ipykernel_33554/2457898955.py:26: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_33554/2457898955.py:26: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_33554/2457898955.py:26: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_33554/2457898955.py:26: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykern

In [15]:
meta_df = meta_df[['SITE_ID', 'LOCATION_LON', 'LOCATION_LAT', 'MIN_DATE', 'MAX_DATE', 'geometry']]
gdf = gpd.GeoDataFrame(meta_df, crs='EPSG:4326', geometry=meta_df['geometry'])
gdf.to_file(DATA_DIR / 'sites.geojson', driver='GeoJSON')